# 02_IBM-HHRR_ATTRITION_Parametro_v3


## Previous Tasks

### Import Libraries

In [1]:
# Generic Libraries
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  RobustScaler
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML  

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score
from sklearn.metrics import  f1_score, fbeta_score, confusion_matrix
from sklearn.model_selection import GridSearchCV

In [2]:
# Define general path:
path_general = r'C:\HHRR'
path_total = os.path.join(path_general,'01_total_models') 

In [3]:
# Model Libraries.

# Cross validation
from sklearn.model_selection import cross_val_score 

#------------- / Regresion Logistica /--------------
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression

#--------------/ XGBoost /--------------------------
from xgboost import XGBClassifier
import xgboost as xgb

#-------------/ AdaBoost /--------------------------
from sklearn.ensemble import AdaBoostClassifier

#-------------/ CatBoost /--------------------------
from catboost import CatBoostClassifier

#------------/ Decission Tree /----------------------
from sklearn.tree import DecisionTreeClassifier

#------------/ Random Forest /-----------------------
from sklearn.ensemble import RandomForestClassifier

#-----------/   MLP /--------------------------------
from sklearn.neural_network import MLPClassifier

#------------/ KNN /----------------------------------
from sklearn.neighbors import KNeighborsClassifier

#------------/ Naive - Bayes /-------------------------
from sklearn.naive_bayes import GaussianNB


### Load Dataset

In [4]:
# Load dataset
df = pd.read_csv(r'C:\Users\TrendingPC\Desktop\PROYECTOS\HHRR-IBM_ATTRITION\HR-Employee-ML.csv', index_col=0)
df

,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,Education_Associate Degree,Education_Bachelor’s Degree,Education_Doctorate,Education_High School,...,PercentSalaryHike,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,annual_salary,economic_impact
0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,11,0,8,0,6,4,0,5,71916,14670.864
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,23,1,10,3,10,7,1,7,61560,12558.240
2,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,15,0,7,3,0,0,0,0,25080,4037.880
3,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,11,0,8,3,8,7,3,0,34908,6876.876
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,12,1,6,3,2,2,2,2,41616,8198.352
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,17,1,17,3,5,2,0,3,30852,6077.844
1466,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,15,1,9,5,7,7,1,7,119892,25177.320
1467,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,20,1,6,0,6,2,0,3,73704,15035.616
1468,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,14,0,17,3,9,6,0,8,64680,13194.720


In [5]:
X = df.drop(columns='Attrition')
y = df['Attrition']

In [6]:
# Separation of the dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42,stratify=y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1029, 78), (441, 78), (1029,), (441,))

In [7]:
# Check dataset composition

print(" Fraudulent Count for Full data :  ",np.sum(y))
print(" Fraudulent Count for Train data : ",np.sum(y_train))
print(" Fraudulent Count for Test data :   ",np.sum(y_test))

 Fraudulent Count for Full data :   237
 Fraudulent Count for Train data :  166
 Fraudulent Count for Test data :    71


In [8]:
# Save the testing set for evaluation
X_test_saved = X_test.copy()
y_test_saved = y_test.copy()
print("Saved X_test & y_test")

Saved X_test & y_test


## 1.- Transformaciones de datos.

## Dataset Original

### Smote

In [9]:
# Import of specific libraries
from collections import Counter
from imblearn.over_sampling import SMOTE

# Initial situation
print('Original dataset shape %s' % Counter(y_train))

# Calculate OverSampling model
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

print('Resampled dataset shape %s' % Counter(y_train_smote))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 863})


### Adasyn

In [10]:
# Import of specific libraries
from imblearn.over_sampling import ADASYN

# Initial situation
print('Original dataset shape %s' % Counter(y_train))

# Calculate OverSampling model
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)

print('Resampled dataset shape %s' % Counter(y_train_adasyn))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 800})


## Power Transformation

### Original

In [11]:
# - Apply : preprocessing.PowerTransformer(copy=False) to fit & transform the train & test data

from sklearn import metrics 
from sklearn import preprocessing


from sklearn.preprocessing import PowerTransformer

pt= preprocessing.PowerTransformer(method='yeo-johnson', copy=True)  # creates an instance of the PowerTransformer class.
pt.fit(X_train)

X_train_pt = pt.transform(X_train)
X_test_pt = pt.transform(X_test)

y_train_pt = y_train
y_test_pt = y_test

### Smote

In [12]:
# Import of specific libraries
from collections import Counter
from imblearn.over_sampling import SMOTE

# Initial situation
print('Original dataset shape %s' % Counter(y_train_pt))

# Calculate OverSampling model
smote = SMOTE(random_state=42)
X_train_smote_pt, y_train_smote_pt = smote.fit_resample(X_train_pt, y_train_pt)

print('Resampled dataset shape %s' % Counter(y_train_smote_pt))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 863})


### Adasyn

In [13]:
# Import of specific libraries
from imblearn.over_sampling import ADASYN

# Initial situation
print('Original dataset shape %s' % Counter(y_train))

# Calculate OverSampling model
adasyn = ADASYN(random_state=42)
X_train_adasyn_pt, y_train_adasyn_pt = adasyn.fit_resample(X_train_pt, y_train_pt)

print('Resampled dataset shape %s' % Counter(y_train_adasyn_pt))

Original dataset shape Counter({0: 863, 1: 166})
Resampled dataset shape Counter({0: 863, 1: 839})


### Load Model: Libraries and Functions.

In [14]:
# LOAD OF MODELS.
# perfom cross validation on the X_train & y_train 
from sklearn.model_selection import StratifiedKFold

# Initialize StratifiedKFold cross-validator
# perform cross validation
skf = StratifiedKFold(n_splits=3, random_state=None, shuffle=False)
#  Shuffle is False because we need a constant best model when we use GridSearchCV


In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict

### Create dataset_list

In [16]:
# Original distribution
OR_origin = ['OR origin',X_train, y_train, X_test, y_test]
OR_smote =['OR smote',X_train_smote, y_train_smote, X_test, y_test]
OR_adasyn = ['OR adasyn', X_train_adasyn, y_train_adasyn, X_test, y_test]

# Power Transformation
PT_origin = ['PT origin',X_train_pt, y_train_pt, X_test_pt, y_test_pt]
PT_smote = ['PT smote',X_train_smote_pt, y_train_smote_pt, X_test_pt, y_test_pt ]
PT_adasyn = ['PT adasyn', X_train_adasyn_pt, y_train_adasyn_pt, X_test_pt, y_test_pt]

### Create models

In [17]:
model_list = ['regression_logistic', 'adaboost', 'xgboost', 'catboost', 'decision_tree', 'random_forest', 'mlp', 'knn']
parameters = [
    [0.1, 0.5, 1, 1.5, 2, 2.5, 3],      # For 'regression_logistic'
    [5, 7, 9],                          # For 'adaboost'
    [0.001, 0.01, 0.1, 0.5, 1, 3],      # For 'xgboost'
    [100, 200, 300, 400, 500, 600],      # For 'catboost'
    [1, 2, 3, 4, 5],                    # For 'decision_tree'
    [100, 200, 400, 600, 800, 1000, 1200],  # For 'random_forest'
    [(50,), (100,), (120,), (150,)],     # For 'mlp'
    [3, 5, 7]  #knn
    ]

In [18]:
distributions =[OR_origin, OR_smote, OR_adasyn, PT_origin, PT_smote, PT_adasyn]
complete_model = zip(model_list, parameters)
complete_model_list = list(complete_model )
complete_model_list

[('regression_logistic', [0.1, 0.5, 1, 1.5, 2, 2.5, 3]),
 ('adaboost', [5, 7, 9]),
 ('xgboost', [0.001, 0.01, 0.1, 0.5, 1, 3]),
 ('catboost', [100, 200, 300, 400, 500, 600]),
 ('decision_tree', [1, 2, 3, 4, 5]),
 ('random_forest', [100, 200, 400, 600, 800, 1000, 1200]),
 ('mlp', [(50,), (100,), (120,), (150,)]),
 ('knn', [3, 5, 7])]

In [19]:
def gen_models(complete_model_list, distributions, save_directory_complete_model=None):
    
    # test if directory is None and add path
    if save_directory_complete_model is None:
        save_directory_complete_model = os.path.join(os.getcwd(), 'total')
        
    #Create folder if not exits
    os.makedirs(save_directory_complete_model, exist_ok=True)

    
    # 1.- Iterate model
    for model_name, param_values in complete_model_list:
        #print(f"Processing model: {model_name}")  
        
        resultados_totales = []  
        
        # 2.- Iterate over distributions:
        for distribution in distributions:
            try:
                # Unpack distribution
                name = distribution[0]  # Nombre de la distribución
                X_train, y_train, X_val, y_val = distribution[1:]

                #print(f"  Distribution: {name}")  # Log distribución

                # Verify if data is valid:
                if X_train is None or y_train is None or X_val is None or y_val is None:
                    print(f"    Skipping due to missing data in {name}")
                    continue
                
                # 3.- Iterate over params:
                for param in param_values:
                    #print(f"    Training {model_name} with parameter {param}")  # Log parámetro

                    # Inicializate model and parameters
                    model_instance = None
                    if model_name == 'regression_logistic':
                        model_instance = LogisticRegression(C=param)
                        parameter_name ='C='
                    elif model_name == 'adaboost':
                        model_instance = AdaBoostClassifier(DecisionTreeClassifier(max_depth=param))
                        parameter_name = 'max_depth='
                        #model_instance = AdaBoostClassifier(DecisionTreeClassifier(iterations=param))
                        #parameter_name = 'iterations='
                    elif model_name == 'xgboost':
                        model_instance = XGBClassifier(learning_rate=param)
                        parameter_name  = 'learning_rate='
                    elif model_name == 'catboost':
                        model_instance = CatBoostClassifier(iterations=param, verbose=0)
                        parameter_name  = 'iterations='
                    elif model_name == 'decision_tree':
                        model_instance = DecisionTreeClassifier(max_depth=param)
                        parameter_name  = 'max_depth='
                    elif model_name == 'random_forest':
                        model_instance = RandomForestClassifier(n_estimators=param)
                        parameter_name  = 'n_estimators= '
                    elif model_name == 'mlp':
                        model_instance = MLPClassifier(hidden_layer_sizes=param)
                        parameter_name  = 'hidden_layer_sizes='
                    elif model_name == 'knn':
                        model_instance = KNeighborsClassifier(n_neighbors=param)
                        parameter_name  = 'n_neighbors='
                    else:
                        print(f"    Invalid model name: {model_name}")
                        continue

                    
                    # Train model
                    model_instance.fit(X_train, y_train)

                    # Get predictions
                    y_pred = model_instance.predict(X_val)

                    # Calculate metrics
                    roc_auc = roc_auc_score(y_val, y_pred)
                    accuracy = accuracy_score(y_val, y_pred)
                    precision = precision_score(y_val, y_pred)
                    recall = recall_score(y_val, y_pred)
                    f1 = f1_score(y_val, y_pred)
                    f2 = fbeta_score(y_val, y_pred, beta=2)
                    confusion = confusion_matrix(y_val, y_pred)

                    # Save results in DataFrame:
                    results_df = pd.DataFrame({
                        'Model': [model_name],
                        'Description': [name],
                        'Parameter':[parameter_name + str(param)],
                        'ROC-AUC': [roc_auc],
                        'Accuracy': [accuracy],
                        'Precision': [precision],
                        'Recall': [recall],
                        'F1 Score': [f1],
                        'F2 Score': [f2],
                        'Confusion Matrix': [confusion],
                    })

                    resultados_totales.append(results_df)

            except Exception as e:
                print(f"  Error processing distribution {name}: {str(e)}")
                continue

        # Save results:
        if resultados_totales:
            df_resultados_final = pd.concat(resultados_totales, ignore_index=True)
            save_path = os.path.join(save_directory_complete_model, f"{model_name}_total.csv")
            df_resultados_final.to_csv(save_path, index=False)
            display(HTML(f"<h2 style='text-align: center;font-size:60px;'> Modelo: {model_name}</h2>")) 
            display(df_resultados_final)
            print(f"\n\n\nResults for {model_name} saved to {save_path}")
        else:
            print(f"No results generated for {model_name}")



In [20]:
gen_models(complete_model_list, distributions, path_total)

,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,regression_logistic,OR origin,C=0.1,0.500000,0.839002,0.000000,0.000000,0.000000,0.000000,"[[370, 0], [71, 0]]"
1,regression_logistic,OR origin,C=0.5,0.508679,0.834467,0.333333,0.028169,0.051948,0.034483,"[[366, 4], [69, 2]]"
2,regression_logistic,OR origin,C=1,0.504340,0.836735,0.333333,0.014085,0.027027,0.017422,"[[368, 2], [70, 1]]"
3,regression_logistic,OR origin,C=1.5,0.505691,0.839002,0.500000,0.014085,0.027397,0.017483,"[[369, 1], [70, 1]]"
4,regression_logistic,OR origin,C=2,0.505691,0.839002,0.500000,0.014085,0.027397,0.017483,"[[369, 1], [70, 1]]"
5,regression_logistic,OR origin,C=2.5,0.505691,0.839002,0.500000,0.014085,0.027397,0.017483,"[[369, 1], [70, 1]]"
6,regression_logistic,OR origin,C=3,0.504340,0.836735,0.333333,0.014085,0.027027,0.017422,"[[368, 2], [70, 1]]"
7,regression_logistic,OR smote,C=0.1,0.564827,0.632653,0.210191,0.464789,0.289474,0.374150,"[[246, 124], [38, 33]]"
8,regression_logistic,OR smote,C=0.5,0.620879,0.659864,0.251572,0.563380,0.347826,0.451467,"[[251, 119], [31, 40]]"
9,regression_logistic,OR smote,C=1,0.588085,0.662132,0.232877,0.478873,0.313364,0.395349,"[[258, 112], [37, 34]]"





Results for regression_logistic saved to C:\HHRR\01_total_models\regression_logistic_total.csv


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,adaboost,OR origin,max_depth=5,0.541759,0.823129,0.360000,0.126761,0.187500,0.145631,"[[354, 16], [62, 9]]"
1,adaboost,OR origin,max_depth=7,0.540122,0.829932,0.400000,0.112676,0.175824,0.131579,"[[358, 12], [63, 8]]"
2,adaboost,OR origin,max_depth=9,0.520061,0.834467,0.400000,0.056338,0.098765,0.068027,"[[364, 6], [67, 4]]"
3,adaboost,OR smote,max_depth=5,0.588352,0.834467,0.470588,0.225352,0.304762,0.251572,"[[352, 18], [55, 16]]"
4,adaboost,OR smote,max_depth=7,0.547164,0.832200,0.428571,0.126761,0.195652,0.147541,"[[358, 12], [62, 9]]"
5,adaboost,OR smote,max_depth=9,0.566654,0.845805,0.578947,0.154930,0.244444,0.181518,"[[362, 8], [60, 11]]"
6,adaboost,OR adasyn,max_depth=5,0.564808,0.814059,0.358974,0.197183,0.254545,0.216718,"[[345, 25], [57, 14]]"
7,adaboost,OR adasyn,max_depth=7,0.580738,0.850340,0.619048,0.183099,0.282609,0.213115,"[[362, 8], [58, 13]]"
8,adaboost,OR adasyn,max_depth=9,0.545527,0.839002,0.500000,0.112676,0.183908,0.133333,"[[362, 8], [63, 8]]"
9,adaboost,PT origin,max_depth=5,0.537419,0.825397,0.363636,0.112676,0.172043,0.130719,"[[356, 14], [63, 8]]"





Results for adaboost saved to C:\HHRR\01_total_models\adaboost_total.csv


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,xgboost,OR origin,learning_rate=0.001,0.500000,0.839002,0.000000,0.000000,0.000000,0.000000,"[[370, 0], [71, 0]]"
1,xgboost,OR origin,learning_rate=0.01,0.541188,0.841270,0.538462,0.098592,0.166667,0.117845,"[[364, 6], [64, 7]]"
2,xgboost,OR origin,learning_rate=0.1,0.601865,0.857143,0.666667,0.225352,0.336842,0.259740,"[[362, 8], [55, 16]]"
3,xgboost,OR origin,learning_rate=0.5,0.628683,0.863946,0.689655,0.281690,0.400000,0.319489,"[[361, 9], [51, 20]]"
4,xgboost,OR origin,learning_rate=1,0.609478,0.841270,0.513514,0.267606,0.351852,0.295950,"[[352, 18], [52, 19]]"
5,xgboost,OR origin,learning_rate=3,0.564522,0.823129,0.393939,0.183099,0.250000,0.205047,"[[350, 20], [58, 13]]"
6,xgboost,OR smote,learning_rate=0.001,0.584583,0.818594,0.395349,0.239437,0.298246,0.259939,"[[344, 26], [54, 17]]"
7,xgboost,OR smote,learning_rate=0.01,0.612181,0.845805,0.542857,0.267606,0.358491,0.297806,"[[354, 16], [52, 19]]"
8,xgboost,OR smote,learning_rate=0.1,0.623278,0.854875,0.606061,0.281690,0.384615,0.315457,"[[357, 13], [51, 20]]"
9,xgboost,OR smote,learning_rate=0.5,0.623278,0.854875,0.606061,0.281690,0.384615,0.315457,"[[357, 13], [51, 20]]"


...




Results for random_forest saved to C:\HHRR\01_total_models\random_forest_total.csv


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,mlp,OR origin,"hidden_layer_sizes=(50,)",0.537648,0.682540,0.200000,0.323944,0.247312,0.288221,"[[278, 92], [48, 23]]"
1,mlp,OR origin,"hidden_layer_sizes=(100,)",0.495946,0.832200,0.000000,0.000000,0.000000,0.000000,"[[367, 3], [71, 0]]"
2,mlp,OR origin,"hidden_layer_sizes=(120,)",0.504625,0.827664,0.222222,0.028169,0.050000,0.034130,"[[363, 7], [69, 2]]"
3,mlp,OR origin,"hidden_layer_sizes=(150,)",0.461191,0.535147,0.135870,0.352113,0.196078,0.267094,"[[211, 159], [46, 25]]"
4,mlp,OR smote,"hidden_layer_sizes=(50,)",0.530510,0.231293,0.169951,0.971831,0.289308,0.500000,"[[33, 337], [2, 69]]"
5,mlp,OR smote,"hidden_layer_sizes=(100,)",0.525885,0.242630,0.168766,0.943662,0.286325,0.491924,"[[40, 330], [4, 67]]"
6,mlp,OR smote,"hidden_layer_sizes=(120,)",0.532927,0.244898,0.170854,0.957746,0.289979,0.498534,"[[40, 330], [3, 68]]"
7,mlp,OR smote,"hidden_layer_sizes=(150,)",0.581005,0.344671,0.188571,0.929577,0.313539,0.520505,"[[86, 284], [5, 66]]"
8,mlp,OR adasyn,"hidden_layer_sizes=(50,)",0.520423,0.777778,0.212766,0.140845,0.169492,0.151057,"[[333, 37], [61, 10]]"
9,mlp,OR adasyn,"hidden_layer_sizes=(100,)",0.521698,0.827664,0.333333,0.070423,0.116279,0.083612,"[[360, 10], [66, 5]]"





Results for mlp saved to C:\HHRR\01_total_models\mlp_total.csv


,Model,Description,Parameter,ROC-AUC,Accuracy,Precision,Recall,F1 Score,F2 Score,Confusion Matrix
0,knn,OR origin,n_neighbors=3,0.497659,0.777778,0.153846,0.084507,0.109091,0.092879,"[[337, 33], [65, 6]]"
1,knn,OR origin,n_neighbors=5,0.505196,0.809524,0.190476,0.056338,0.086957,0.065574,"[[353, 17], [67, 4]]"
2,knn,OR origin,n_neighbors=7,0.518995,0.823129,0.294118,0.070423,0.113636,0.083056,"[[358, 12], [66, 5]]"
3,knn,OR smote,n_neighbors=3,0.524496,0.603175,0.179012,0.408451,0.248927,0.325112,"[[237, 133], [42, 29]]"
4,knn,OR smote,n_neighbors=5,0.506357,0.591837,0.165644,0.380282,0.230769,0.302013,"[[234, 136], [44, 27]]"
5,knn,OR smote,n_neighbors=7,0.521013,0.578231,0.175141,0.436620,0.250000,0.336226,"[[224, 146], [40, 31]]"
6,knn,OR adasyn,n_neighbors=3,0.507994,0.585034,0.166667,0.394366,0.234310,0.309735,"[[230, 140], [43, 28]]"
7,knn,OR adasyn,n_neighbors=5,0.502589,0.575964,0.162791,0.394366,0.230453,0.307018,"[[226, 144], [43, 28]]"
8,knn,OR adasyn,n_neighbors=7,0.515607,0.569161,0.171271,0.436620,0.246032,0.333333,"[[220, 150], [40, 31]]"
9,knn,PT origin,n_neighbors=3,0.552569,0.841270,0.529412,0.126761,0.204545,0.149502,"[[362, 8], [62, 9]]"





Results for knn saved to C:\HHRR\01_total_models\knn_total.csv
